## Libraries

In [511]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
import struct
import math
import datetime
import sys
import struct

## Data

In [515]:
occup = pd.read_csv('occupancy_6005.csv', parse_dates=[0], index_col=[0])# данные заполненности дорог
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   sensor_1  500 non-null    int64
 1   sensor_2  500 non-null    int64
 2   sensor_3  500 non-null    int64
 3   sensor_4  500 non-null    int64
 4   sensor_5  500 non-null    int64
dtypes: int64(5)
memory usage: 19.7 KB


In [516]:
occup.head()

,value
timestamp,
2015-09-01 13:45:00,3.06
2015-09-01 13:50:00,6.44
2015-09-01 13:55:00,5.17
2015-09-01 14:00:00,3.83
2015-09-01 14:05:00,4.50


In [506]:
speed = pd.read_csv('speed_6005.csv', parse_dates=[0], index_col=[0])# данные скорости
speed.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2500 entries, 2015-08-31 18:22:00 to 2015-09-17 16:24:00
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   value   2500 non-null   int64
dtypes: int64(1)
memory usage: 39.1 KB


In [507]:
speed.head()

,value
timestamp,
2015-08-31 18:22:00,90
2015-08-31 18:32:00,80
2015-08-31 18:57:00,84
2015-08-31 19:07:00,94
2015-08-31 19:12:00,90


Подготовим данные для тестирования алгоритма Sprintz. Из одного ряда получим датафрейм с 5 рядами.

In [508]:
speed['index'] = np.arange(speed.shape[0])
sen_0 = speed.loc[speed['index']%5==0, ['value']]['value'].to_list()
sen_1 = speed.loc[speed['index']%5==1, ['value']]['value'].to_list()
sen_2 = speed.loc[speed['index']%5==2, ['value']]['value'].to_list()
sen_3 = speed.loc[speed['index']%5==3, ['value']]['value'].to_list()
sen_4 = speed.loc[speed['index']%5==4, ['value']]['value'].to_list()
print(len(sen_0))
print(len(sen_1))
print(len(sen_2))
print(len(sen_3))
print(len(sen_4))

500
500
500
500
500


In [509]:
df = pd.DataFrame({'sensor_1': sen_0, 'sensor_2': sen_1, 'sensor_3': sen_2, 'sensor_4': sen_3, 'sensor_5': sen_4})
df.head()

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
0,90,80,84,94,90
1,91,62,96,75,84
2,86,93,69,88,71
3,74,68,88,71,87
4,82,68,73,69,57


In [510]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   sensor_1  500 non-null    int64
 1   sensor_2  500 non-null    int64
 2   sensor_3  500 non-null    int64
 3   sensor_4  500 non-null    int64
 4   sensor_5  500 non-null    int64
dtypes: int64(5)
memory usage: 19.7 KB


## XOR

In [512]:
#Функция получения XOR двух значений
def fxor(a, b):
    rtrn = []
    a = struct.pack('d', a)
    b = struct.pack('d', b)
    for ba, bb in zip(a, b):
        rtrn.append(ba ^ bb)
    return bin(int.from_bytes(bytes(rtrn), sys.byteorder))[2:].zfill(64)

In [513]:
#Функция представления числа в формате binary-64
def binary(num):
    num = bytes(struct.pack('d', num))
    return bin(int.from_bytes(num, sys.byteorder))[2:].zfill(64)

In [514]:
def xor_compres(ts):
    result = []
    s = binary(ts[0])
    result.append(s)
    off_pred = s.index('1')
    len_pred = len(s)-s[::-1].index('1')-1-off_pred
    for i in range(1, len(ts)):
        xor = fxor(ts[i-1], ts[i])
        #print(xor)
        try:
            off = xor.index('1')#начало ненулевого кода xor
        except:
            s = '0'
            result.append(s)
            continue
        length = len(xor)-xor[::-1].index('1')-1-off# длина последовательности - 1
        if (off<off_pred) or (off+length>off_pred+len_pred) or (i==1):
            offset_b = bin(off)[2:]
            len_seq_b = bin(length)[2:]
            s = '11'+('0'*(5-len(offset_b))+offset_b+'0'*(6-len(len_seq_b))
                      +len_seq_b+xor[off:off+length+1])
            result.append(s)
            off_pred = off
            len_pred = length
        else:
            s = '10'+'0'*(off-off_pred)+xor[off:off+length+1]+'0'*(off_pred+len_pred-length-off)
            result.append(s)
    return result

Так как алгоритм для чисел с плавающей точкой, то разделим значения первого датасета на 100, чтобы сделать их по модулю меньше 1.

In [517]:
occup.value = occup.value/100

In [518]:
time_series = list(occup.value.values)
res = cor_compres(time_series)

Вычислим разницу между исходными данными в формате binary-64 и сжатыми данными.

In [520]:
storage = 0
init_storage = 0
for t in time_series:
    init_storage+=len(binary(t))
print('Размер передаваемого сообщения(бит):', init_storage)
for r in res:
    storage+=len(r)
print('Размер передаваемого сообщения(бит):', storage)
print('Разница:',init_storage-storage) 

Размер передаваемого сообщения(бит): 152320
Размер передаваемого сообщения(бит): 148921
Разница: 3399


## Timestamp delta of delta

In [524]:
occup.index[:6].to_list()

[Timestamp('2015-09-01 13:45:00'),
 Timestamp('2015-09-01 13:50:00'),
 Timestamp('2015-09-01 13:55:00'),
 Timestamp('2015-09-01 14:00:00'),
 Timestamp('2015-09-01 14:05:00'),
 Timestamp('2015-09-01 14:20:00')]

In [522]:
#delta_of_delta = -256
def delta_of_delta_timestamp(a, b, c):
    delta_of_delta = int((c - b).total_seconds() - (b - a).total_seconds())
    if delta_of_delta==0:
        s = '0'
    else:
        delta_b =  bin(abs(delta_of_delta))[2:]
        sgn = '1' if delta_of_delta<0 else '0'
        if -63<=delta_of_delta<=64:
            if delta_of_delta==64:
                s = '10' + delta_b
            else:
                s = '10' + sgn + '0'*(7-len(delta_b)-1) + delta_b
        elif -255<=delta_of_delta<=256:
            if delta_of_delta==256:
                s = '110' + delta_b
            else:
                s = '110' + sgn + '0'*(9-len(delta_b)-1) + delta_b
        elif -2047<=delta_of_delta<=2048:
            if delta_of_delta==2048:
                s = '1110' + delta_b
            else:
                s = '1110' + sgn + '0'*(12-len(delta_b)-1) + delta_b
        else:
            s = '1111' + sgn + '0'*(32-len(delta_b)-1) + delta_b
    return s

In [528]:
def time_compress(t):
    res = []
    t1 = t[0]
    t2 = t[1]
    for i in range(2, len(t)):
        t3 = t[i]
        res.append(delta_of_delta_timestamp(t1, t2, t3))
        t1 = t[i-1]
        t2 = t[i]
    return res

In [529]:
t = occup.index[:6].to_list()
time_compress(t)

['0', '0', '0', '1110001001011000']

## Spritz

In [480]:
def delta_coding(df):
    delta = []
    for i in range(1, df.shape[0]):
        delta.append((df.iloc[i]-df.iloc[i-1]).to_list())
    return pd.DataFrame(delta)

In [481]:
def zigzag_coding(df):
    res = []
    for i in range(df.shape[0]):
        raw = []
        for j in (df.columns):
            val = df.iloc[i][j]
            if val<0:
                raw.append(abs(val)*2-1)
            else:
                raw.append(val*2)
        res.append(raw)
    return res

In [500]:
def sprintz(df):
    enc = [df.iloc[0].to_list()]
    delta = delta_coding(df)
    zigzag = zigzag_coding(delta)
    max_val = list(map(max, zip(*zigzag)))
    max_val = [format(m, '08b') for m in max_val]
    max_len = [8-(m.find('1')) if '1' in m else 0 for m in max_val]
    header = ''.join([format(m, '03b') for m in max_len])
    zigzag = [list(row) for row in zip(*zigzag)] #transpose
    bin = ''
    for i in range(len(zigzag)):
        bits = max_len[i]
        if bits==0:
            continue
        for v in zigzag[i]:
            bin+=format(v, f'0{bits}b')
    enc.append(header+'0'*(16-len(header))+bin)
    return enc

In [501]:
def sprintz_encode(df):
    res = []
    chunk_size = 8
    for i in range(0, len(df), chunk_size):
        chunk_df = df.iloc[i:i+chunk_size]
        res.append(spritz(chunk_df))
    return res

In [531]:
res = sprintz_encode(df)
infb = 0
for r in res:
    infb+=len(r)
print('Объем сжатых данных(бит):', infb)

Объем сжатых данных(бит): 13271
